In [1]:
from kif_lib import Search
from kbel.disambiguators import Disambiguator
# import logging
# logging.basicConfig(level=logging.DEBUG)

### Using `simple` disambiguator to link entities from Wikidata

In [2]:
from kbel.disambiguators.simple import SimpleDisambiguator
kbel = Disambiguator(disambiguator_name='simple')
results = kbel.disambiguate_item(
    label='Rock',
    searcher=Search('wikidata-wapi', limit=10))
display (*results)

('rock music',
 'popular music genre',
 Item(IRI('http://www.wikidata.org/entity/Q11399')))

### Using `similarity` disambiguator to link entities from Wikidata

In [3]:
from kbel.disambiguators.similarity import SimilarityDisambiguator
kbel = Disambiguator(disambiguator_name='sim')
results = kbel.disambiguate_item(
    label='Rock',
    searcher=Search('wikidata-wapi', limit=10),
    sentence='Rock is a stone',
    limit=1)
display (*results)

('stone',
 'rock; building material',
 Item(IRI('http://www.wikidata.org/entity/Q22731')))

### Using `LLM` disambiguator to link entities from Wikidata

Instantiating LLM Disambiguator with IBM WatsonX's models

In [4]:
import os
import dotenv
dotenv.load_dotenv()

from kbel.disambiguators.llm import LLM_Disambiguator

kbel = Disambiguator(
    'llm',
    model_name='meta-llama/llama-3-3-70b-instruct',
    model_provider='ibm',
    model_apikey=os.environ['LLM_API_KEY'],
    model_endpoint=os.environ['LLM_API_ENDPOINT'],
    model_params={
        'project_id': os.environ['WATSONX_PROJECT_ID']
    }
)

In [6]:
results = kbel.disambiguate_item(
    label='Rock',
    searcher=Search('wikidata-wapi', limit=100),
    sentence='A rock can be used in construction to mimic the appearance and durability of natural stone.')

display (*results)

('rock',
 'mass of stone projecting out of the ground or water',
 Item(IRI('http://www.wikidata.org/entity/Q1404150')))

('rock',
 'naturally occurring solid aggregate of one or more minerals or mineraloids',
 Item(IRI('http://www.wikidata.org/entity/Q8063')))

LLM disambiguator uses [LangChain ChatModels](https://python.langchain.com/docs/integrations/providers/). Below, we use IBM WatsonX's models

In [7]:
from langchain_ibm import ChatWatsonx
model = ChatWatsonx(
    model_id='meta-llama/llama-3-3-70b-instruct',
    apikey=os.environ['LLM_API_KEY'], # type: ignore
    url=os.environ['LLM_API_ENDPOINT'], # type: ignore
    project_id=os.environ['WATSONX_PROJECT_ID'],
    temperature=0.0
)
kbel = Disambiguator(disambiguator_name='llm', model=model)

In [8]:
results = kbel.disambiguate_item(
    label='Rock',
    searcher=Search('wikidata-wapi', limit=100),
    sentence='A rock can be used in construction to mimic the appearance and durability of natural stone.')
display (*results)

('rock',
 'mass of stone projecting out of the ground or water',
 Item(IRI('http://www.wikidata.org/entity/Q1404150')))

('rock',
 'naturally occurring solid aggregate of one or more minerals or mineraloids',
 Item(IRI('http://www.wikidata.org/entity/Q8063')))

Disambiguating properties:

In [9]:
results = kbel.disambiguate_property(
    label='instance of', searcher=Search('wikidata-wapi', limit=20), sentence='John is a Human')
display(*results)

('instance of',
 'type to which this subject corresponds/belongs. Different from P279 (subclass of); for example: K2 is an instance of mountain; volcanoes form a subclass of mountains',
 Property(IRI('http://www.wikidata.org/entity/P31'), None))

Linking entities to DBpedia

In [10]:
searcher = Search('dbpedia', limit=100)
results = kbel.disambiguate_item(label='Rock', searcher=searcher, sentence='A rock can be used in construction to mimic the appearance and durability of natural stone.')
display(*results)

('Metamorphic rock',
 '',
 Item(IRI('http://dbpedia.org/resource/Metamorphic_rock')))

('Sedimentary rock',
 '',
 Item(IRI('http://dbpedia.org/resource/Sedimentary_rock')))

Linking entities to PubChem

In [11]:
results = kbel.disambiguate_item(label='benzene', searcher=Search('pubchem-ddgs', limit=20), sentence='A benzene can be used in construction to mimic the appearance and durability of natural stone.')
display(*results)

('PubChem Benzene | C6H6 | CID 241 - PubChem',
 'Benzene | C6H6 | CID 241 - structure, chemical names, physical and chemical properties, classification, patents, literature, biological activities, safety/hazards/toxicity information, supplier lists, and more.',
 Item(IRI('http://rdf.ncbi.nlm.nih.gov/pubchem/compound/CID241')))